## Request and Inspect IWM-Options

The Russell 2000 ETF (IWM) is a popular instrument for conservative option strategies.

In this example, we select call options suitible for a neutral/ moderate baerish option selling strategy


First, load ib-ruby environment

In [1]:
require_relative '../../lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

Define the base

In [2]:
iwm =  IB::Stock.new( symbol: :iwm ).verify.first
puts iwm.as_table

┌───────┬────────┬─────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│       │ symbol │ con_id  │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞═══════╪════════╪═════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Stock │ IWM    │ 9579970 │  SMART   │        │            │      IWM      │       │        │   USD    │
└───────┴────────┴─────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘


In [3]:
iwm_market_price =  iwm.market_price
puts "IWM: Bars (raw-data): #{iwm.bars}  \n    -->  market_price #{iwm.misc}"


IWM: Bars (raw-data): [{}]  
    -->  market_price {:delayed=>nil}


* We are requesting market data in a demo-account without realtime data permissions.  
* Concurrent calls of `iwm.market_price` add further raw-data tuples to `iwm.bars`. However, no timestamp is saved.
         
If the request is fired before the market opening, the returned market price is the closing price of the previous day. 


In [4]:
iwm_options = iwm.otm_options(right: :call, count: 10)
puts "Out of the Money Strikes to consider:" 
puts iwm_options.keys.to_f.join ", "

Out of the Money Strikes to consider:
225.0, 226.0, 227.0, 228.0, 229.0, 230.0, 231.0, 232.0, 233.0, 234.0, 235.0


In [5]:
puts "Available Expiries: "
puts iwm_options[iwm_options.keys.max].as_table

Available Expiries: 
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ IWM    │ 607346865 │ IBUSOPT  │ 20230421 │    100     │      IWM      │ call  │  235.0 │   USD    │
│ Option │ IWM    │ 586093224 │ IBUSOPT  │ 20230519 │    100     │      IWM      │ call  │  235.0 │   USD    │
│ Option │ IWM    │ 537473997 │ IBUSOPT  │ 20230616 │    100     │      IWM      │ call  │  235.0 │   USD    │
│ Option │ IWM    │ 613637859 │ IBUSOPT  │ 20230721 │    100     │      IWM      │ call  │  235.0 │   USD    │
│ Option │ IWM    │ 603702084 │ IBUSOPT  │ 20230818 │    100     │      IWM      │ call  │  235.0 │   USD    │
│ Option │ IWM    │ 578275528 │ IBUSOPT  │ 20230915 │    100     │      IWM      │ call  │ 

In [6]:
puts iwm_options.map{|x,y| y.detect{|o| o.strike == iwm_options.keys[5]}}.compact.as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ IWM    │ 607346835 │ IBUSOPT  │ 20230421 │    100     │      IWM      │ call  │  230.0 │   USD    │
└────────┴────────┴───────────┴──────────┴──────────┴────────────┴───────────────┴───────┴────────┴──────────┘


Lets inspect these Options further.

The method _request_greeks_ requests Tickdata from the TWS-Server and collects the calculated greeks.

The parameter `thread: true` switches to asynchronous requests. The results are always stored in `Option.greek`,
which is an alias for the `OptionDetail`-Model. Thus calling `.as_table` on `Option,greek` prints the contents of this model. 


In [9]:
iwm_options.each do  |x,y| 
   # update contracts with greeks 
   y.map{|o| o.request_greeks( thread: true)}.join;
   # and print them
   puts y.compact.sort_by( &:expiry).greek.as_table  unless y.compact.size.zero?
end
nil   # suppress further output

┌───────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                    │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞═══════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ call  IWM 20230421  225.0 │    0.00 │       0.25 │     0.00 │    0.00 │    0.00 │    0.00 │   -0.00 │
│ call  IWM 20230519  225.0 │    0.01 │       0.23 │     0.00 │    0.00 │    0.00 │    0.01 │   -0.00 │
│ call  IWM 20230616  225.0 │    0.02 │       0.19 │     0.00 │    0.01 │    0.00 │    0.01 │   -0.00 │
│ call  IWM 20230721  225.0 │    0.06 │       0.18 │     0.54 │    0.01 │    0.00 │    0.03 │   -0.00 │
│ call  IWM 20230818  225.0 │    0.14 │       0.18 │     0.54 │    0.02 │    0.00 │    0.06 │   -0.00 │
│ call  IWM 20230915  225.0 │    0.26 │       0.17 │     0.54 │    0.03 │    0.00 │    0.10 │   -0.01 │
│ call  IWM 20231117  225.0 │    0.66 │       0.17 │     1.01 │ 